In [47]:
import numpy as np
import tensorflow
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import math

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [6]:
MODEL_PATH = '/saved_models/nlp_models'

In [7]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 32.7 MB/s 
     |████████████████████████████████| 233 kB 61.4 MB/s 
     |████████████████████████████████| 1.4 MB 16.4 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394487 sha256=fa6736b2e7da860bba6fc63fac6b0855db009d69ea598987c714e149ea0f0669
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=153428 sha256=b6d8a381141e1643b7481abb94a2fd9164fca3cb2bfd5495336e810b73493194
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7


###**Text preprocessing functions**

In [8]:
from __future__ import unicode_literals
from hazm import *
import re
import random
from string import punctuation

def text_preprocess(text):
  normalizer = Normalizer()
  text = normalizer.normalize(text)
  text = re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)
  return text

def text_scramble(text):
  words = text.split()
  random.shuffle(words)
  return ' '.join(words)


###**Loading and processing Corpus 1**
(VOA FARSI 2003-08)

In [9]:
path = 'data/voa_fa_2003-2008_orig.txt'
with open(path, 'r', encoding='utf-8') as f:
  text = ' '.join([line.strip() for line in f.readlines() if not line.startswith('#')])
  text = text.split('.')
  sents = random.sample(text, 25000)

In [10]:
MAX_LEN = 170
sents_cleaned = [text_preprocess(s) for s in sents]
sents_ready = filter(lambda s: len(s) < MAX_LEN, sents_cleaned)
sents_data = list(sents_ready)
random.shuffle(sents_data)
sents_data = sents_data[:20000]  # make dataset size 20000 for ease of computation

In [11]:
tokenizer = keras.preprocessing.text.Tokenizer(oov_token='<UNK>')
tokenizer.fit_on_texts(sents_data)
tokenizer.word_index['<PAD>'] = 0
last_idx = len(tokenizer.word_index) + 1
tokenizer.word_index['<SOS>'] = last_idx

In [12]:
temp = [len(each.split()) for each in sents_data]
print('maximum sentence length:', sorted(temp)[-1])

maximum sentence length: 42


### **building the dataset**

In [13]:
def create_shuffles(sent_list, m):
  """
  this function creates m random shuffles of the sentence
  """
  all_combs = []
  for sent in sent_list:
    comb_set = set([])
    for i in range(0, m + 1):
      comb_set.add(text_scramble(sent))
    all_combs.append((sent, comb_set))
  return all_combs

In [14]:
shuffle_pairs = create_shuffles(sents_data, 10)

In [15]:
shuffle_pairs[1][0], shuffle_pairs[1][1]

(' یک تصادف رانندگی به علت شدت باد و دود ناشی از آتش سوزی در بزرگراه ۱۱۸ باعث مرگ راننده این اتومبیل و مجروح شدن سرنشینان دیگر شد',
 {'اتومبیل ۱۱۸ در و علت مجروح سرنشینان راننده از دود و باعث ناشی یک مرگ شدن تصادف این باد شد دیگر بزرگراه رانندگی به آتش شدت سوزی',
  'این ۱۱۸ سوزی بزرگراه علت در آتش به دیگر یک ناشی شدت سرنشینان از و مرگ رانندگی و اتومبیل باعث باد شدن شد راننده تصادف دود مجروح',
  'باعث ناشی در دیگر سوزی شدت دود این از آتش مجروح علت و سرنشینان شد باد شدن رانندگی مرگ بزرگراه به تصادف راننده یک ۱۱۸ و اتومبیل',
  'بزرگراه اتومبیل و باد سرنشینان دود باعث و به یک از تصادف شدت آتش شد این دیگر ناشی رانندگی شدن در راننده ۱۱۸ مجروح مرگ سوزی علت',
  'در سوزی آتش دیگر و بزرگراه باد شدت و به مجروح تصادف یک ۱۱۸ مرگ شد ناشی شدن دود علت این رانندگی سرنشینان باعث راننده از اتومبیل',
  'در یک تصادف رانندگی باعث اتومبیل بزرگراه علت و سوزی شدت شد شدن آتش باد ناشی دود از مرگ دیگر و ۱۱۸ سرنشینان راننده مجروح این به',
  'راننده دود دیگر علت مرگ از ناشی سرنشینان شد به باعث رانندگی این آتش و سوز

In [16]:
def create_dataframe(pairs):
  """
  creates a dataframe with two columns of original sentence and it's shuffles
  """
  original = []
  shuffled = []
  for pair in pairs:
    for p in pair[1]:
      original.append(pair[0])
      shuffled.append(p)

  df_dict = { 'Shuffled': shuffled, 'Original': original}
  df = pd.DataFrame(df_dict)
  return df


In [17]:
df = create_dataframe(shuffle_pairs)
df

,Shuffled,Original
0,است آسیب حاکی کرده خبری شوهر بازداشت داده است ...,گزارش‌های خبری حاکی است پلیس شوهر زن نروژی را...
1,کرده شوهر حاکی گزارش‌های مورد نروژی و زن را ند...,گزارش‌های خبری حاکی است پلیس شوهر زن نروژی را...
2,گزارش‌های که ندید است بازجوئی خبری بازداشت را ...,گزارش‌های خبری حاکی است پلیس شوهر زن نروژی را...
3,مورد زن شوهر ندید نروژی بازجوئی را است که قرار...,گزارش‌های خبری حاکی است پلیس شوهر زن نروژی را...
4,داده گزارش‌های است آسیب است مورد و زن شوهر حاک...,گزارش‌های خبری حاکی است پلیس شوهر زن نروژی را...
...,...,...
217021,شد دوباره به نفر آوردن مقام آغاسی اول پیرترین ...,با بدست آوردن دوباره این مقام آغاسی جایگزین ...
217022,این بدست مقام به شد پیرترین نفر آوردن جایگزین ...,با بدست آوردن دوباره این مقام آغاسی جایگزین ...
217023,با عنوان به در مقام دوباره تنیس کانرز پیرترین ...,با بدست آوردن دوباره این مقام آغاسی جایگزین ...
217024,اول کانرز تنیس دنیا آوردن به جیمی شد عنوان دوب...,با بدست آوردن دوباره این مقام آغاسی جایگزین ...


In [18]:
def create_dataset(df, train_percent=80):
  data_size = df.shape[0]
  count = math.floor(data_size * (train_percent/100))

  dataset = df['Shuffled'].values
  labels  = df['Original'].values

  train_data   = dataset[:count]
  train_labels = labels[:count]
  test_data   = dataset[count:]
  test_labels = labels[count:]

  return ((train_data, train_labels), (test_data, test_labels))




In [19]:
train_set, test_set = create_dataset(df)
X_train_text, Y_train_text = train_set
X_test_text, Y_test_text = test_set

###**making dataset ready for training**

In [20]:
sos_index = last_idx
def shift_output_sequence(seq_list):
  for seq in seq_list:
    seq.insert(0, sos_index)
  return seq_list


In [21]:
X_train_seq = tokenizer.texts_to_sequences(X_train_text)
Y_train_seq = tokenizer.texts_to_sequences(Y_train_text)
X_test_seq  = tokenizer.texts_to_sequences(X_test_text)
Y_test_seq  = tokenizer.texts_to_sequences(Y_test_text)

In [22]:
MAX_SEQ_LEN = 45  
X_train = keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=MAX_SEQ_LEN)
Y_train = keras.preprocessing.sequence.pad_sequences(Y_train_seq, maxlen=MAX_SEQ_LEN)

X_test  = keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen=MAX_SEQ_LEN)
Y_test  = keras.preprocessing.sequence.pad_sequences(Y_test_seq, maxlen=MAX_SEQ_LEN)

X_train_decoder = keras.preprocessing.sequence.pad_sequences(shift_output_sequence(Y_train_seq), maxlen=MAX_SEQ_LEN)
X_test_decoder  = keras.preprocessing.sequence.pad_sequences(shift_output_sequence(Y_test_seq),  maxlen=MAX_SEQ_LEN)

In [23]:
X_train_decoder[:2], X_test_decoder[:2]

(array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0, 24248,   873,   380,
           494,     8,    73,  4432,   451,  4131,     6,     7,   553,
          3627,   202,    29,     4,    40,  1758,    47,    89,     8],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0, 24248,   873,   380,
           494,     8,    73,  4432,   451,  4131,     6,     7,   553,
          3627,   202,    29,     4,    40,  1758,    47,    89,     8]],
       dtype=int32),
 array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0, 24248,   586,
           234,  1708,     2,  2507,   5

In [137]:
tokenizer.sequences_to_texts(X_train[:1]), tokenizer.sequences_to_texts(Y_train[:1])

(['<UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> است آسیب حاکی کرده خبری شوهر بازداشت داده است ندید زن را مورد قرار گزارش\u200cهای و بازجوئی نروژی پلیس که'],
 ['<UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> گزارش\u200cهای خبری حاکی است پلیس شوهر زن نروژی را که آسیب ندید بازداشت کرده و مورد بازجوئی قرار داده است'])

In [25]:
VOCAB_SIZE = len(tokenizer.word_index)
VOCAB_SIZE

24248

###**Encoder-Decoder network**

In [26]:
encoder_embedding_size = 32
decoder_embedding_size = 32
lstm_units = 128

np.random.seed(42)
tf.random.set_seed(42)

encoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)

encoder_embedding = keras.layers.Embedding(input_dim=VOCAB_SIZE + 1,output_dim=encoder_embedding_size, input_length=MAX_SEQ_LEN, mask_zero=True)(encoder_input)

_, encoder_state_h, encoder_state_c = keras.layers.LSTM(lstm_units, dropout=0.2, return_state=True, recurrent_regularizer='l2')(encoder_embedding)

encoder_state = [encoder_state_h, encoder_state_c]

decoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)

decoder_embedding = keras.layers.Embedding(input_dim=VOCAB_SIZE + 2, output_dim=decoder_embedding_size, mask_zero=True)(decoder_input)

decoder_lstm_output = keras.layers.LSTM(lstm_units, dropout=0.2, return_sequences=True, recurrent_regularizer='l2')(decoder_embedding, initial_state=encoder_state)

decoder_output = keras.layers.Dense(VOCAB_SIZE + 1, activation="softmax")(decoder_lstm_output)

model = keras.models.Model(inputs=[encoder_input, decoder_input], outputs=[decoder_output])

optimizer = keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 32)     775968      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 32)     776000      ['input_2[0][0]']                
                                                                                              

In [28]:
history = model.fit([X_train, X_train_decoder], Y_train, epochs=30, validation_split=0.2, batch_size=512)

Epoch 1/30
272/272 [==============================] - 184s 676ms/step - loss: 3.4998 - accuracy: 0.0782 - val_loss: 3.0202 - val_accuracy: 0.0808
Epoch 2/30
272/272 [==============================] - 183s 673ms/step - loss: 2.7111 - accuracy: 0.1130 - val_loss: 2.4847 - val_accuracy: 0.1860
Epoch 3/30
272/272 [==============================] - 183s 672ms/step - loss: 2.0178 - accuracy: 0.3285 - val_loss: 1.7223 - val_accuracy: 0.5233
Epoch 4/30
272/272 [==============================] - 183s 673ms/step - loss: 1.3267 - accuracy: 0.6116 - val_loss: 1.1621 - val_accuracy: 0.7422
Epoch 5/30
272/272 [==============================] - 183s 673ms/step - loss: 0.8829 - accuracy: 0.7679 - val_loss: 0.8712 - val_accuracy: 0.8351
Epoch 6/30
272/272 [==============================] - 183s 672ms/step - loss: 0.6256 - accuracy: 0.8386 - val_loss: 0.7197 - val_accuracy: 0.8758
Epoch 7/30
272/272 [==============================] - 184s 677ms/step - loss: 0.4590 - accuracy: 0.8806 - val_loss: 0.6352 -

**almost 100% accuracy on training data after 30 epochs**

In [33]:
model.save("sentence_model_v1.h5")

###**Simple seq2seq network**

**model evaluation on test data**

In [48]:
model = keras.models.load_model("sentence_model_v1.h5")

In [49]:
ids = np.argmax(model.predict([X_test[:1], X_test_decoder[:1]]), axis=-1)
ids, Y_test[:1]

(array([[5651, 5651, 5651, 5651, 5651, 5651, 5651, 5651, 5651, 5651, 5651,
         5651, 5651, 5651, 5651, 5651, 5651, 5651, 5651, 5651, 5651, 5651,
         5651, 5651, 5651,    0,  586,  234, 1708,    2, 2507,  530,   79,
         5155,    2,   31, 9010,    3,   11,  918, 2980, 2980,   43,  737,
           23]]),
 array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,   586,
           234,  1708,     2,  2507,   530,    79,  5155,     2,    31,
          9010,     3,    11,   918,  2980, 21714,    43,   737,    23]],
       dtype=int32))

In [68]:
print("Evaluate on test data")
results = model.evaluate([X_test[:10], X_test_decoder[:10]], Y_test[:10], batch_size=512)
print("test loss, test acc:", results)

Evaluate on test data
1/1 [==============================] - 0s 54ms/step - loss: 0.6553 - accuracy: 0.9442
test loss, test acc: [0.6552770137786865, 0.9441624283790588]


###**Rebuild sentences**

In [120]:
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)

def remove_padding_and_join(texts):
  for i in range(len(texts)):
    no_padding = filter(lambda a: a != '<PAD>', texts[i])
    texts[i] = ' '.join(list(no_padding))
  return texts
    

def reconstruct_predicted_list(texts):
  for i in range(len(texts)):
    pad_start = texts[i].index('<PAD>')
    texts[i] = texts[i][pad_start:]
    no_padding = filter(lambda a: a != '<PAD>', texts[i])
    texts[i] = ' '.join(list(no_padding))
  return texts

def get_predictions_on_test(count):
    Y_preds = model.predict([X_test[:count], X_test_decoder[:count]])
    encoded_argmax  = np.argmax(Y_preds, axis=-1)
    return encoded_argmax, Y_test[:count]

def get_predicted_sentences(predicted):
  predicted_word_list = list(map(sequence_to_text, predicted))
  predicted_sent_list = reconstruct_predicted_list(predicted_word_list)
  return predicted_sent_list

def get_original_sentences(original):
  original_word_list  = list(map(sequence_to_text, original))
  original_sent_list = remove_padding_and_join(original_word_list)
  return original_sent_list 


def rebuild_sentence_from_sequence(count=20):
  predicted, original = get_predictions_on_test(count)
  predicted_sent_list = get_predicted_sentences(predicted)
  original_sent_list  = get_original_sentences(original)
  return predicted_sent_list, original_sent_list
  


In [122]:
preds, origs = rebuild_sentence_from_sequence()
preds, origs

(['آخرین بار آرژانتین در بازی\u200cهای المپیک شهر هلسینکی در سال ۱۹۵۲ به یک مدال طلای طلای دست یافته بود',
  'آخرین بار آرژانتین در بازی\u200cهای المپیک شهر هلسینکی در سال ۱۹۵۲ به یک مدال طلای طلای دست یافته بود',
  'آخرین بار آرژانتین در بازی\u200cهای المپیک شهر هلسینکی در سال ۱۹۵۲ به یک مدال طلای طلای دست یافته بود',
  'آخرین بار آرژانتین در بازی\u200cهای المپیک شهر هلسینکی در سال ۱۹۵۲ به یک مدال طلای طلای دست یافته بود',
  'آخرین بار آرژانتین در بازی\u200cهای المپیک شهر هلسینکی در سال ۱۹۵۲ به یک مدال طلای طلای دست یافته بود',
  'آخرین بار آرژانتین در بازی\u200cهای المپیک شهر هلسینکی در سال ۱۹۵۲ به یک مدال طلای طلای دست یافته بود',
  'آخرین بار آرژانتین در بازی\u200cهای المپیک شهر هلسینکی در سال ۱۹۵۲ به یک مدال طلای طلای دست یافته بود',
  'آخرین بار آرژانتین در بازی\u200cهای المپیک شهر هلسینکی در سال ۱۹۵۲ به یک مدال طلای طلای دست یافته بود',
  'آخرین بار آرژانتین در بازی\u200cهای المپیک شهر هلسینکی در سال ۱۹۵۲ به یک مدال طلای طلای دست یافته بود',
  'انفجارروز رئیس کمیسیون انتخابات نی

In [159]:
def predict_a_sentence(sentence):
  seq = tokenizer.texts_to_sequences([sentence])
  X = keras.preprocessing.sequence.pad_sequences(seq, maxlen=MAX_SEQ_LEN)
  X_decoder = keras.preprocessing.sequence.pad_sequences(shift_output_sequence(seq), maxlen=MAX_SEQ_LEN)
  Y_pred = model.predict([X, X_decoder])
  encoded_argmax  = np.argmax(Y_pred, axis=-1)

  output = get_predicted_sentences(encoded_argmax)
  return output


In [163]:
print(predict_a_sentence(X_train_text[0]))
print(Y_train_text[0])
print(X_train_text[0])

print('----')

print('shuffled: \n', X_test[:1], '\n')
print('predicted: \n', np.argmax(model.predict([X_test[:1], X_test_decoder[:1]]), axis=-1), '\n')
print('original: \n', Y_test[:1], '\n')

['است آسیب حاکی کرده خبری شوهر بازداشت داده است ندید زن را مورد قرار گزارش\u200cهای و بازجوئی نروژی پلیس که']
 گزارش‌های خبری حاکی است پلیس شوهر زن نروژی را که آسیب ندید بازداشت کرده و مورد بازجوئی قرار داده است
است آسیب حاکی کرده خبری شوهر بازداشت داده است ندید زن را مورد قرار گزارش‌های و بازجوئی نروژی پلیس که
----
shuffled: 
 [[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0   737    11     2    43    31    23  1708 21714   586  2980
    918    79     2   234  5155  2507   530  9010     3]] 

predicted: 
 [[5651 5651 5651 5651 5651 5651 5651 5651 5651 5651 5651 5651 5651 5651
  5651 5651 5651 5651 5651 5651 5651 5651 5651 5651 5651    0  586  234
  1708    2 2507  530   79 5155    2   31 9010    3   11  918 2980 2980
    43  737   23]] 

original: 
 [[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0   